<a href="https://colab.research.google.com/github/LordV98/MagnesiumSiteSearcher/blob/main/MagnesiumSiteSearcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instala biopython se necessário
!pip install biopython

from Bio.PDB import PDBParser
from math import sqrt
from google.colab import files
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

# Entrada interativa para o espaçamento no eixo X
step = int(input("🔢 Intervalo entre rótulos no eixo X (ex: 1 para todos, 5 para espaçar): "))

# Conversão de resíduos
three_to_one = {
    "ALA": "A", "ARG": "R", "ASN": "N", "ASP": "D", "CYS": "C", "GLN": "Q", "GLU": "E",
    "GLY": "G", "HIS": "H", "ILE": "I", "LEU": "L", "LYS": "K", "MET": "M", "PHE": "F",
    "PRO": "P", "SER": "S", "THR": "T", "TRP": "W", "TYR": "Y", "VAL": "V"
}

# Padrões usados no preditor
padroes_pontuados = {
    'ASP+0 ASP+1': 89, 'GLU+0 ASP+1': 46, 'ASP+0 GLU+1': 38,
    'ASP+0 ASP+1 ASP+2': 35, 'GLU+0 GLU+1': 24, 'ASN+0 ASP+1': 22,
    'ASP+0 ASN+1': 20, 'HIS+0 GLU+1': 18, 'SER+0 THR+1': 15,
    'ASP+0 GLU+1 ASP+2': 14, 'ASP+0 SER+1': 13, 'HIS+0 HIS+1': 13,
    'ARG+0 ASP+1': 12, 'GLU+0 SER+1': 12, 'GLU+0 ARG+1': 11,
    'SER+0 GLU+1': 11, 'ASP+0 HIS+1': 11, 'THR+0 THR+1': 11,
    'GLU+0 HIS+1': 11, 'SER+0 ASP+1': 11
}

def distancia(a, b):
    return sqrt(sum((a[i] - b[i])**2 for i in range(3)))

def centroide(coords):
    x, y, z = zip(*coords)
    return (sum(x)/len(x), sum(y)/len(y), sum(z)/len(z))

def filtrar_atom_only(original_path):
    atom_path = original_path.replace(".pdb", "_atom.pdb")
    with open(original_path, "r") as f:
        linhas = f.readlines()
    atom_linhas = [l for l in linhas if l.startswith("ATOM") or l.startswith("HETATM") or l.startswith("END")]
    with open(atom_path, "w") as f:
        f.writelines(atom_linhas)
    return atom_path

def analisar_pdb(arquivo_pdb, cadeia='A'):
    atom_only = filtrar_atom_only(arquivo_pdb)
    parser = PDBParser(QUIET=True, PERMISSIVE=True)
    estrutura = parser.get_structure("estrutura", atom_only)
    modelo = estrutura[0]

    if cadeia not in modelo:
        print(f"❌ Cadeia '{cadeia}' não encontrada no arquivo '{arquivo_pdb}'.")
        print("🔧 Para continuar, edite o arquivo .pdb e rotule a cadeia desejada como 'A'.")
        return []

    cadeia = modelo[cadeia]
    sequencia = []
    posicoes = []
    for res in cadeia:
        if res.id[0] == ' ':
            aa = three_to_one.get(res.resname, "X")
            if aa != "X":
                sequencia.append(aa)
                posicoes.append(res)

    resultados = []
    for i in range(len(sequencia)):
        for padrao, peso in padroes_pontuados.items():
            partes = padrao.split()
            match = True
            indices = []
            coords = []

            for parte in partes:
                tipo, offset = parte.split('+')
                pos = i + int(offset)
                if pos >= len(sequencia) or three_to_one.get(tipo, "X") != sequencia[pos]:
                    match = False
                    break
                try:
                    ca = posicoes[pos]['CA'].get_coord()
                    coords.append(ca)
                    indices.append(posicoes[pos].id[1])
                except:
                    match = False
                    break

            if match:
                centro = centroide(coords)
                resultados.append({
                    "padrao": padrao,
                    "residuos": indices,
                    "centroide": centro
                })

    return resultados

def plotar_distancias_centroides(resultados):
    distancias = []
    pares = []
    dados_csv = []
    centroides = [r['centroide'] for r in resultados]
    pontos_medios = []

    for i in range(len(resultados)):
        for j in range(i + 1, len(resultados)):
            d = distancia(centroides[i], centroides[j])
            distancias.append(d)
            par = f"{i+1}-{j+1}"
            pares.append(par)
            dados_csv.append({"Par de motivos": par, "Distância (Å)": round(d, 3)})
            pm = tuple(np.mean([centroides[i], centroides[j]], axis=0))
            pontos_medios.append(pm)

    plt.figure(figsize=(12, 6))
    plt.bar(pares, distancias)
    plt.xticks(ticks=np.arange(0, len(pares), step), labels=pares[::step], rotation=90)
    plt.xlabel("Par de motivos")
    plt.ylabel("Distância entre centroides (Å)")
    plt.title("Distâncias entre centroides de motivos detectados")
    plt.tight_layout()
    plt.show()

    df_dist = pd.DataFrame(dados_csv)
    df_dist.to_csv("/content/distancias_centroides.csv", index=False)
    files.download("/content/distancias_centroides.csv")

    df_pm = pd.DataFrame(pontos_medios, columns=['X', 'Y', 'Z'])
    df_pm["Par de motivos"] = pares
    df_pm.to_csv("/content/pontos_medios_centroides.csv", index=False)
    files.download("/content/pontos_medios_centroides.csv")

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    for i, c in enumerate(centroides):
        ax.scatter(*c, color='blue')
        ax.text(*c, str(i+1), color='black')
    for i in range(len(centroides)):
        for j in range(i + 1, len(centroides)):
            x = [centroides[i][0], centroides[j][0]]
            y = [centroides[i][1], centroides[j][1]]
            z = [centroides[i][2], centroides[j][2]]
            ax.plot(x, y, z, color='gray', alpha=0.5)
    for p in pontos_medios:
        ax.scatter(*p, color='red', s=50)
    ax.set_title("Centroides e Pontos Médios entre Motivos")
    plt.tight_layout()
    plt.show()

# Execução principal
uploaded = files.upload()
if uploaded:
    nome_arquivo = list(uploaded.keys())[0]
    resultados = analisar_pdb(nome_arquivo)
    for r in resultados:
        print(f"Motivo: {r['padrao']}")
        print(f"Resíduos: {r['residuos']}")
        print(f"Centroide: {tuple(round(c,2) for c in r['centroide'])}\\n")
    plotar_distancias_centroides(resultados)
else:
    print("❌ Nenhum arquivo enviado.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 32.2 MB/s eta 0:00:00
